In [1]:
import sqlite3
connect = sqlite3.connect('spider.sqlite')
con = connect.cursor()

In [2]:
print 'Create json file named data.js'
num = int(raw_input('Number of Nodes:'))

Create json file named data.js
Number of Nodes:30


In [3]:
con.execute('''SELECT COUNT(from_id) AS in_link, old_rank, new_rank, id, url 
                FROM Pages JOIN Links ON Pages.id = Links.to_id 
                WHERE html IS NOT NULL AND ERROR IS NULL
                GROUP BY id ORDER BY id,in_link''')

In [4]:
handle = open('data.js', 'w')
nodes, max_rank, min_rank = [], 0, float('inf')
for line in con:
    nodes.append(line)
    rank = line[2]
    max_rank = max(max_rank, rank)
    min_rank = min(min_rank, rank)
    if len(nodes) > num: break
if max_rank == min_rank or max_rank ==0 or min_rank == float('inf'):
    print 'Error - No valid page rank'
    quit()

In [7]:
handle.write('dataJson = {"nodes":[\n')
count, mp, ranks = 0, {}, {}
for line in nodes:
    if count > 0: handle.write(',\n')
    rank = line[2]
    rank = (num-1) * ( (rank - min_rank) / (max_rank - min_rank) ) # normalization
    handle.write('{'+'"weight":'+str(line[0])+',"rank":'+str(rank)+',"id":'+str(line[3])+', "url":"'+line[4]+'"}')
    mp[line[3]], ranks[line[3]] = count, rank
    count += 1
handle.write('],\n') 

In [ ]:
con.execute('''SELECT DISTINCT from_id, to_id FROM Links''')
handle.write('"links":[\n')

In [ ]:
count = 0
for line in con:
    if line[0] not in mp or line[1] not in mp: continue
    if count > 0: handle.write(',\n')
    rank = ranks[line[0]]
    nrank = (num-1)*( (rank - minrank) / (maxrank - minrank) )
    handle.write('{"source":'+str(map[row[0]])+',"target":'+str(map[row[1]])+',"value":'+str(nrank)+'}')
    count += 1
handle.write(']};')
handle.close()
con.close()

print "Open force.html in a browser to view the visualization"